In [2]:
from sklearn.model_selection import KFold
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dropout, Flatten, Dense,Input,BatchNormalization, Activation,MaxPooling2D
from tensorflow.keras.optimizers import RMSprop,Adam,SGD
from tensorflow.keras.regularizers import l2,l1
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Add
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [3]:
tf.__version__

'2.4.0'

# Data Pre-Processing

Open **play_style_train.csv** file and split the games into a list.
Every row of csv: `PSL0000000001,1,B[pd],W[dp],B[qp],W[dc],B[nq],W[nc],B[qf],W[kd],B[ce],W[dg],B[dd],W[cc],B[fd],W[ed],B[ee],W[ec],B[ge],W[gc],B[di]`. 

Columns are:

    1. PSL0000000001: Game ID
    2. 1: Game Style
    3-... : Moves, the last move represents the play style (B[di] in this case)
    
We cropped only the moves to game list as:

In [4]:
df = open('/home/ttsai/AIcupTutorial-main/Training Dataset/play_style_train.csv').read().splitlines()
games = [i.split(',',2)[-1] for i in df]
game_styles = [int(i.split(',',2)[-2]) for i in df]


Create a dictionary to convert the coordinates from characters to numbers

In [5]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
coordinates

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18}

We decided to build a DCNN model in this tutorial. We create data samples by using every move in every game, meaning that the target is to predict the next move by feeding the previous state of the table in every game for every move. Therefore, we can collect much more data samples from games.

For the simplicity, we used 2 dimensional feature map to represent the data as below:
 1. Occupied areas: mark them as 1 and the empty places as 0
 2. The last move in the table: mark the position of the last move as 1 and the rest as 0
 
The target is to predict the game style (1, 2 or 3) from the state of the game table. Later this will be one-hot encoded.

In [ ]:
def prepare_input(moves):
    x = np.zeros((19,19,2))
    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        x[row,column,0] = 1
    if moves:
        last_move_column = coordinates[moves[-1][2]]
        last_move_row = coordinates[moves[-1][3]]
        x[row,column,1] = 1
    return x

In [6]:
def generate_custom_features(ownermap):
    features = []

    # 特徵1：白棋位置
    white_features = np.zeros((19, 19))
    white_features[ownermap == 'O'] = 1
    features.append(white_features)

    # 特徵2：黑棋位置
    black_features = np.zeros((19, 19))
    black_features[ownermap == 'X'] = 1
    features.append(black_features)

    # 特徵3：所有棋子位置
    occupied_features = np.zeros((19, 19))
    occupied_features[(ownermap == 'O') | (ownermap == 'X')] = 1
    features.append(occupied_features)

    # 特徵4：黑棋下一步可能被包圍的位置
    # 需要你的邏輯來判斷
    # 這裡使用示例來標記可能被包圍的位置
    black_surrounded = np.zeros((19, 19))
    black_surrounded[1, 3] = 1
    features.append(black_surrounded)

    # 特徵5：白棋下一步可能被包圍的位置
    # 需要你的邏輯來判斷
    # 這裡使用示例來標記可能被包圍的位置
    white_surrounded = np.zeros((19, 19))
    white_surrounded[2, 2] = 1
    features.append(white_surrounded)

    return np.array(features)
# 添加連線特徵
def check_lines(ownermap):
    # 假設連線特徵包括水平、垂直和對角線方向的連線
    connected_lines = np.zeros((19, 19))

    # 檢查水平連線
    for i in range(19):
        for j in range(16):
            if ownermap[i, j] == ownermap[i, j + 1] == ownermap[i, j + 2]:
                connected_lines[i, j] = connected_lines[i, j + 1] = connected_lines[i, j + 2] = 1

    # 檢查垂直連線
    for i in range(16):
        for j in range(19):
            if ownermap[i, j] == ownermap[i + 1, j] == ownermap[i + 2, j]:
                connected_lines[i, j] = connected_lines[i + 1, j] = connected_lines[i + 2, j] = 1

    # 檢查對角線（往右上角）
    for i in range(16):
        for j in range(16):
            if ownermap[i, j] == ownermap[i + 1, j + 1] == ownermap[i + 2, j + 2]:
                connected_lines[i, j] = connected_lines[i + 1, j + 1] = connected_lines[i + 2, j + 2] = 1

    # 檢查對角線（往右下角）
    for i in range(3, 19):
        for j in range(16):
            if ownermap[i, j] == ownermap[i - 1, j + 1] == ownermap[i - 2, j + 2]:
                connected_lines[i, j] = connected_lines[i - 1, j + 1] = connected_lines[i - 2, j + 2] = 1

    return connected_lines

# 添加可能的攻擊或防守位置
def evaluate_positions(ownermap):
    # 假設這裡的策略是找到敵方下一步可能落子的位置作為攻擊，並找到自己下一步可能落子的位置作為防守
    possible_attacks = np.zeros((19, 19))
    possible_defenses = np.zeros((19, 19))

    for i in range(19):
        for j in range(19):
            if ownermap[i, j] == 0:
                # 模擬在這個位置下一步敵方落子後的局勢
                temp_ownermap = np.copy(ownermap)
                temp_ownermap[i, j] = 2  # 假設對手為白色
                # 假設這個位置下一步能夠形成連線
                if np.any(check_lines(temp_ownermap)):
                    possible_attacks[i, j] = 1

                # 模擬在這個位置下一步自己落子後的局勢
                temp_ownermap = np.copy(ownermap)
                temp_ownermap[i, j] = 1  # 假設自己為黑色
                # 假設這個位置下一步能夠形成連線
                if np.any(check_lines(temp_ownermap)):
                    possible_defenses[i, j] = 1

    return possible_attacks, possible_defenses



In [8]:
def prepare_input(moves, ownermap):
    x = np.zeros((19, 19, 8))  # 加入新的特徵通道

    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        x[row, column, 0] = 1  # 第一個通道保留原有的特徵

        # 更新 ownermap
        ownermap[row][column] = 1 if color == 'B' else 2

    # 加入特徵根到第三個通道
    for i in range(19):
        for j in range(19):
            if ownermap[i][j] == 1:
                x[i, j, 2] = 1  # 黑子的特徵根
            elif ownermap[i][j] == 2:
                x[i, j, 3] = 1  # 白子的特徵根

    if moves:
        last_move_column = coordinates[moves[-1][2]]
        last_move_row = coordinates[moves[-1][3]]
        x[last_move_row, last_move_column, 1] = 1  # 最後一步的特徵

        # 調用 generate_custom_features 函數並加入到 x 中
        new_features = generate_custom_features(ownermap)
        for i in range(5):  # 假設生成了5個新特徵
            x[:, :, 3 + i] = new_features[i]

    return x


In [10]:
def prepare_input(moves, ownermap):
    x = np.zeros((19, 19, 13))  # 增加新的特徵通道數至 13

    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        x[row, column, 0] = 1  # 第一個通道保留原有的特徵

        # 更新 ownermap
        ownermap[row][column] = 1 if color == 'B' else 2

    # 加入特徵根到第三個通道
    for i in range(19):
        for j in range(19):
            if ownermap[i][j] == 1:
                x[i, j, 2] = 1  # 黑子的特徵根
            elif ownermap[i][j] == 2:
                x[i, j, 3] = 1  # 白子的特徵根

    if moves:
        last_move_column = coordinates[moves[-1][2]]
        last_move_row = coordinates[moves[-1][3]]
        x[last_move_row, last_move_column, 1] = 1  # 最後一步的特徵

        # 調用 generate_custom_features 函數並加入到 x 中
        new_features = generate_custom_features(ownermap)
        
        # 將新特徵依序添加到 x 中
        for i in range(5):  # 假設生成了5個新特徵
            x[:, :, 4 + i] = new_features[i]

        # 添加連線、攻擊和防守特徵
        connected_lines = check_lines(ownermap)
        possible_attacks, possible_defenses = evaluate_positions(ownermap)
        
        x[:, :, 9] = connected_lines  # 連線特徵
        x[:, :, 10] = possible_attacks  # 攻擊特徵
        x[:, :, 11] = possible_defenses  # 防守特徵

    return x


In [11]:
# Check how many samples can be obtained
n_games = 0
for game in games:
    n_games += 1
print(f"Total Games: {n_games}")

Total Games: 26615


Since play style training has smaller dataset comparing to kyu or dan training, we can put the complete dataset to memory. Still, it is better to create a data generator.

In [6]:
x = []
for game in games:
    moves_list = game.split(',')
    x.append(prepare_input(moves_list))
x = np.array(x)
y = np.array(game_styles)-1

In [12]:
x = []
y = []
ownermap = np.zeros((19, 19))  # 初始化 ownermap
for game, style in zip(games, game_styles):
    moves_list = game.split(',')
    x.append(prepare_input(moves_list, ownermap))
    # 基於遊戲類型來生成標籤
    # 在這裡，假設game_styles中的數字1, 2, 3分別對應標籤0, 1, 2
    y.append(style - 1)

x = np.array(x)
y = np.array(y)


/tmp/ipykernel_62016/1982268569.py:6: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  white_features[ownermap == 'O'] = 1
/tmp/ipykernel_62016/1982268569.py:11: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  black_features[ownermap == 'X'] = 1
/tmp/ipykernel_62016/1982268569.py:16: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  occupied_features[(ownermap == 'O') | (ownermap == 'X')] = 1


In [13]:
x.shape

(26615, 19, 19, 13)

In [14]:
y.shape

(26615,)

In [15]:
np.bincount(y)

array([8184, 9403, 9028])

In [16]:
y_hot = tf.one_hot(y, depth=3)

2023-11-28 21:19:23.609507: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-11-28 21:19:23.609993: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-11-28 21:19:23.612804: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-28 21:19:23.612884: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3060 computeCapability: 8.6
coreClock: 1.837GHz coreCount: 28 deviceMemorySize: 11.74GiB deviceMemoryBandwidth: 335.32GiB/s
2023-11-28 21:19:23.612908: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA nod

Dataset splitting: 90% Training, 10% validation

In [17]:
x_train, x_val, y_train, y_val = train_test_split(x, y_hot.numpy(), test_size=0.10)

# Training

### Simple DCNN Model:

In [38]:
def create_new_regularized_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(19, 19, 8)))
    #model.add(Dropout(0.05))  # Dropout 正則化
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    #model.add(Dropout(0.3))  # Dropout 正則化
    model.add(Flatten())
    model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.005)))  # L2 正則化
    #model.add(Dropout(0.05))  # Dropout 正則化
    model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.005)))  # L2 正則化
    #model.add(Dropout(0.2))  # Dropout 正則化
    model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.005)))  # L2 正則化
    #model.add(Dropout(0.2))  # Dropout 正則化
    model.add(Dense(3, activation='softmax'))

    opt = RMSprop(learning_rate=0.0003)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [66]:
def create_regularized_model_v2():
    model = Sequential()
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(19, 19, 13)))
    model.add(Dropout(0.05))  # 增加了 Dropout 正則化
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(Dropout(0.05))  # 增加了 Dropout 正則化
    model.add(Flatten())
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))  # 增加了 L2 正則化
    model.add(Dropout(0.03))  # 增加了 Dropout 正則化
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))  # 增加了 L2 正則化
    model.add(Dropout(0.02))  # 增加了 Dropout 正則化
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))  # 增加了 L2 正則化
    model.add(Dropout(0.03))  # 增加了 Dropout 正則化
    model.add(Dense(3, activation='softmax'))

    opt = RMSprop(learning_rate=0.0002)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    return model
"""Epoch 1/35
375/375 [==============================] - 2s 6ms/step - loss: 2.3900 - accuracy: 0.3892 - val_loss: 1.5117 - val_accuracy: 0.4519
Epoch 2/35
375/375 [==============================] - 2s 5ms/step - loss: 1.4147 - accuracy: 0.4388 - val_loss: 1.3083 - val_accuracy: 0.3937
Epoch 3/35
375/375 [==============================] - 2s 5ms/step - loss: 1.1603 - accuracy: 0.4590 - val_loss: 1.1029 - val_accuracy: 0.4523
Epoch 4/35
375/375 [==============================] - 2s 5ms/step - loss: 1.0945 - accuracy: 0.4672 - val_loss: 1.0889 - val_accuracy: 0.4354
Epoch 5/35
375/375 [==============================] - 2s 5ms/step - loss: 1.0598 - accuracy: 0.4818 - val_loss: 1.1101 - val_accuracy: 0.4230
Epoch 6/35
375/375 [==============================] - 2s 5ms/step - loss: 1.0404 - accuracy: 0.5010 - val_loss: 1.0456 - val_accuracy: 0.5222
Epoch 7/35
375/375 [==============================] - 2s 5ms/step - loss: 1.0150 - accuracy: 0.5403 - val_loss: 0.9468 - val_accuracy: 0.6112
Epoch 8/35
375/375 [==============================] - 2s 5ms/step - loss: 0.9349 - accuracy: 0.6146 - val_loss: 0.9186 - val_accuracy: 0.6150
Epoch 9/35
375/375 [==============================] - 2s 5ms/step - loss: 0.8938 - accuracy: 0.6337 - val_loss: 0.8812 - val_accuracy: 0.6281
Epoch 10/35
375/375 [==============================] - 2s 5ms/step - loss: 0.8649 - accuracy: 0.6442 - val_loss: 0.8919 - val_accuracy: 0.6210
Epoch 11/35
375/375 [==============================] - 2s 5ms/step - loss: 0.8485 - accuracy: 0.6576 - val_loss: 0.8636 - val_accuracy: 0.6450
Epoch 12/35
375/375 [==============================] - 2s 5ms/step - loss: 0.8321 - accuracy: 0.6623 - val_loss: 0.8727 - val_accuracy: 0.6360
Epoch 13/35
...
Epoch 34/35
375/375 [==============================] - 2s 5ms/step - loss: 0.5922 - accuracy: 0.8271 - val_loss: 0.9486 - val_accuracy: 0.6559
Epoch 35/35
375/375 [==============================] - 2s 5ms/step - loss: 0.5880 - accuracy: 0.8324 - val_loss: 0.9568 - val_accuracy: 0.6600

SyntaxError: EOF while scanning triple-quoted string literal (3346469464.py, line 49)

In [71]:
def create_regularized_model_v2():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(19, 19, 13)))
    model.add(Dropout(0.05))  # 增加了 Dropout 正則化
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(Dropout(0.05))  # 增加了 Dropout 正則化
    model.add(Flatten())
    model.add(Dense(16, activation='relu', kernel_regularizer=l2(0.01)))  # 增加了 L2 正則化
    model.add(Dropout(0.03))  # 增加了 Dropout 正則化
    model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))  # 增加了 L2 正則化
    model.add(Dropout(0.02))  # 增加了 Dropout 正則化
    model.add(Dense(16, activation='relu', kernel_regularizer=l2(0.01)))  # 增加了 L2 正則化
    model.add(Dropout(0.03))  # 增加了 Dropout 正則化
    model.add(Dense(3, activation='softmax'))

    opt = RMSprop(learning_rate=0.0002)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [72]:
model = create_regularized_model_v2()
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 17, 17, 32)        3776      
_________________________________________________________________
dropout_71 (Dropout)         (None, 17, 17, 32)        0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 15, 15, 64)        18496     
_________________________________________________________________
dropout_72 (Dropout)         (None, 15, 15, 64)        0         
_________________________________________________________________
flatten_14 (Flatten)         (None, 14400)             0         
_________________________________________________________________
dense_57 (Dense)             (None, 16)                230416    
_________________________________________________________________
dropout_73 (Dropout)         (None, 16)              

In [73]:
history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=64,
    epochs=15,
    validation_data=(x_val, y_val)
)


Epoch 1/15
375/375 [==============================] - 2s 3ms/step - loss: 1.5749 - accuracy: 0.3816 - val_loss: 1.3679 - val_accuracy: 0.3618
Epoch 2/15
375/375 [==============================] - 1s 3ms/step - loss: 1.2823 - accuracy: 0.4208 - val_loss: 1.2430 - val_accuracy: 0.3742
Epoch 3/15
375/375 [==============================] - 1s 3ms/step - loss: 1.1644 - accuracy: 0.4457 - val_loss: 1.1209 - val_accuracy: 0.4361
Epoch 4/15
375/375 [==============================] - 1s 3ms/step - loss: 1.1068 - accuracy: 0.4563 - val_loss: 1.1190 - val_accuracy: 0.4271
Epoch 5/15
375/375 [==============================] - 1s 3ms/step - loss: 1.0744 - accuracy: 0.4663 - val_loss: 1.0827 - val_accuracy: 0.4602
Epoch 6/15
375/375 [==============================] - 1s 3ms/step - loss: 1.0546 - accuracy: 0.4827 - val_loss: 1.0635 - val_accuracy: 0.4557
Epoch 7/15
375/375 [==============================] - 1s 3ms/step - loss: 1.0401 - accuracy: 0.4876 - val_loss: 1.0443 - val_accuracy: 0.4639
Epoch 

In [74]:
model.save('./model_playstyle_2.h5')